    \item k-UpCCGSD ansatz with $k\in\{1; 2; 3\}$, L-BFGS-B optimizer, Jordan-Wigner mapping, Aer simulator,

        \item circuit depth,
    \item gate fidelity,
    \item parameter count,
    \item TRAINABILITY,
    \item EXPRESSIBILITY.

     $H_2$, $He$, $LiH$, $BeH_2$ molecules.
     
     \item converged ground state energy graph, depending on the inter-atomic distance,
\item accuracy, with reference of FCI calculations, absolute error (\parencite{sims2006high}), depending on the inter-atomic distance,
\item number of iterations needed to achieve convergence (gradient tolerance is $10^{-5}$), depending on the inter-atomic distance,
\item for one inter-atomic distance, graph of convergence,
\item runtime, depending on the inter-atomic distance.

> Data from kUpCCGSD-tequila-env.ipynb, or to be precise: kupccgsd-hamiltonians.txt and kupccgsd-ansatzes.txt

In [ ]:
import re
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate

In [ ]:
molecules = [
    ("H2", lambda d: f"H 0.0 0.0 0.0\nH 0.0 0.0 {d}"),
    ("LiH", lambda d: f"Li 0.0 0.0 0.0\nH 0.0 0.0 {d}"),
    ("BeH2", lambda d: f"Be 0.0 0.0 0.0\nH 0.0 0.0 {d}\nH 0.0 0.0 {-d}"),
]

distances=[0.1, 0.2, 0.5, 0.735, 1.0, 1.5, 2.0, 3.0, 4.0]

ks=[1,2,3]

In [ ]:
hamiltonians=[]
hamiltonian_prep_times=[]

ansatzes=[]
ansatz_prep_times=[]

In [ ]:
# example
hamiltonian_str = """
+4.9081+0.2909Z(0)-0.0056Y(0)Z(1)Y(2)-0.0056X(0)Z(1)X(2)+0.2909Z(1)-0.0056Y(1)Z(2)Y(3)-0.0056X(1)Z(2)X(3)-0.7411Z(2)-0.7411Z(3)+0.2481Z(0)Z(1)+0.0781Y(0)Y(2)+0.0781X(0)X(2)+0.0781Z(0)Y(1)Z(2)Y(3)+0.0781Z(0)X(1)Z(2)X(3)+0.0629Y(0)X(1)X(2)Y(3)-0.0629Y(0)Y(1)X(2)X(3)-0.0629X(0)X(1)Y(2)Y(3)+0.0629X(0)Y(1)Y(2)X(3)+0.1693Z(0)Z(2)+0.2322Z(0)Z(3)+0.0836Y(0)Z(1)Y(2)Z(3)+0.0836X(0)Z(1)X(2)Z(3)+0.2322Z(1)Z(2)+0.0836Y(1)Y(3)+0.0836X(1)X(3)+0.1693Z(1)Z(3)+0.2329Z(2)Z(3)
"""

pattern = r'([+-]?\d+\.\d+)([XYZ\(\)\d]*)'
terms = re.findall(pattern, hamiltonian_str)

paulis = []
coeffs = []

for coeff, ops in terms:
    pauli = ['I'] * 4
    for op, idx in re.findall(r'([XYZ])\((\d)\)', ops):
        pauli[int(idx)] = op
    paulis.append(''.join(reversed(pauli)))  # Qiskit uses little-endian
    coeffs.append(float(coeff))

sparse_op = SparsePauliOp(paulis, coeffs)
qc = QuantumCircuit(4)
qc.append(PauliEvolutionGate(sparse_op), range(4))

print(qc)